### Read the configuration

In [10]:
from elasticsearch import Elasticsearch, helpers
import configparser

config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

### Instantiate the Elasticsearch Connection

In [11]:
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['username'], config['ELASTIC']['password'])
)

C:\Users\krish\AppData\Local\Temp/ipykernel_22832/4245316990.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [12]:
es.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': '4f0d5e78d4674150a3d4a00acf8edf38', 'cluster_uuid': 'MtkvfqtcS0O91iJ_V12gDw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [60]:
import pandas as pd

similarity_df = pd.read_csv('sim_index.csv')
similarity_df.head()

,ref_img,sim_img_1,sim_img_2,sim_img_3,sim_img_4,sim_img_5,sim_img_6,sim_img_7,sim_img_8,sim_img_9,sim_img_10
0,patterned_blouse_0.jpg,cutout_back_top_6.jpg,patterned_blouse_106.jpg,patterned_blouse_65.jpg,sweater_26.jpg,patterned_blouse_36.jpg,cutout_back_top_29.jpg,patterned_blouse_74.jpg,patterned_blouse_18.jpg,joggers_7.jpg,sweater_12.jpg
1,patterned_blouse_2.jpg,patterned_blouse_85.jpg,hoodie_0.jpg,hoodie_5.jpg,hoodie_16.jpg,hoodie_9.jpg,patterned_blouse_17.jpg,hoodie_38.jpg,hoodie_18.jpg,hoodie_27.jpg,hoodie_21.jpg
2,patterned_blouse_5.jpg,patterned_blouse_79.jpg,patterned_blouse_68.jpg,patterned_blouse_87.jpg,patterned_blouse_97.jpg,patterned_blouse_24.jpg,patterned_blouse_8.jpg,cutout_back_top_25.jpg,patterned_blouse_106.jpg,patterned_blouse_1.jpg,patterned_blouse_56.jpg
3,patterned_blouse_1.jpg,patterned_blouse_76.jpg,patterned_blouse_5.jpg,patterned_blouse_24.jpg,patterned_blouse_68.jpg,cutout_back_top_25.jpg,patterned_blouse_56.jpg,patterned_blouse_8.jpg,patterned_blouse_87.jpg,patterned_blouse_79.jpg,patterned_blouse_51.jpg
4,patterned_blouse_3.jpg,henley_dress_0.jpg,sweater_11.jpg,sweater_14.jpg,patterned_blouse_101.jpg,sweater_7.jpg,sweater_15.jpg,henley_dress_14.jpg,sweater_4.jpg,sweater_20.jpg,sweater_29.jpg


### Convert dataframe to JSON format

In [61]:
similarity_df = similarity_df.to_dict('records')

In [62]:
similarity_df[1]

{'ref_img': 'patterned_blouse_2.jpg',
 'sim_img_1': 'patterned_blouse_85.jpg',
 'sim_img_2': 'hoodie_0.jpg',
 'sim_img_3': 'hoodie_5.jpg',
 'sim_img_4': 'hoodie_16.jpg',
 'sim_img_5': 'hoodie_9.jpg',
 'sim_img_6': 'patterned_blouse_17.jpg',
 'sim_img_7': 'hoodie_38.jpg',
 'sim_img_8': 'hoodie_18.jpg',
 'sim_img_9': 'hoodie_27.jpg',
 'sim_img_10': 'hoodie_21.jpg'}

### Converting data into appropriate format that elasticsearch can understand

In [63]:
def generator(df):
    for i, line in enumerate(df):
        yield {
            '_index' : 'adm-assignment2',
            '_id' : line.get("ref_img", None),
            '_source' : {
                "sim_img_1": line.get("sim_img_1",""),
                "sim_img_2": line.get("sim_img_2",""),
                "sim_img_3": line.get("sim_img_3",""),
                "sim_img_4": line.get("sim_img_4",""),
                "sim_img_5": line.get("sim_img_5",""),
                "sim_img_6": line.get("sim_img_6",""),
                "sim_img_7": line.get("sim_img_7",""),
                "sim_img_8": line.get("sim_img_8",""),
                "sim_img_9": line.get("sim_img_9",""),
                "sim_img_10": line.get("sim_img_10","")
            }
        }

In [64]:
mycustom = generator(similarity_df)
mycustom

<generator object generator at 0x0000025E2530CC80>

### Settings

In [65]:
Settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "sim_img_1": {
                "type" : "text"
            },
            "sim_img_2": {
                "type" : "text"
            },
            "sim_img_3": {
                "type" : "text"
            },
            "sim_img_4": {
                "type" : "text"
            },
            "sim_img_5": {
                "type" : "text"
            },
            "sim_img_6": {
                "type" : "text"
            },
            "sim_img_7": {
                "type" : "text"
            },
            "sim_img_8": {
                "type" : "text"
            },
            "sim_img_9": {
                "type" : "text"
            },
            "sim_img_10": {
                "type" : "text"
            }
        }
    }
}

### Creating new index with my Elasticsearch instance

In [66]:
IndexName = ''
my_ref_images = es.indices.create( index = 'adm-assignment2', ignore = [400, 404], body = Settings)

C:\Users\krish\AppData\Local\Temp/ipykernel_22832/1387066208.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  my_ref_images = es.indices.create( index = 'adm-assignment2', ignore = [400, 404], body = Settings)
C:\Users\krish\AppData\Local\Temp/ipykernel_22832/1387066208.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  my_ref_images = es.indices.create( index = 'adm-assignment2', ignore = [400, 404], body = Settings)


In [ ]:
my_ref_images

### Bulk upload datafame

In [68]:
from elasticsearch import helpers

In [69]:
try: 
    res = helpers.bulk(es, generator(similarity_df))
    print("Working...")
    print("Response: ", res)
except Exception as e:
    print(e)
    pass

Working...
Response:  (275, [])


### Searching data in the index

In [71]:
result = es.search(
    index = 'adm-assignment2',
    query = {
         "ids": {
             "values": "patterned_blouse_0.jpg" 
         }
    }
)

result['hits']['hits']

[{'_index': 'adm-assignment2',
  '_id': 'patterned_blouse_0.jpg',
  '_score': 1.0,
  '_source': {'sim_img_1': 'cutout_back_top_6.jpg',
   'sim_img_2': 'patterned_blouse_106.jpg',
   'sim_img_3': 'patterned_blouse_65.jpg',
   'sim_img_4': 'sweater_26.jpg',
   'sim_img_5': 'patterned_blouse_36.jpg',
   'sim_img_6': 'cutout_back_top_29.jpg',
   'sim_img_7': 'patterned_blouse_74.jpg',
   'sim_img_8': 'patterned_blouse_18.jpg',
   'sim_img_9': 'joggers_7.jpg',
   'sim_img_10': 'sweater_12.jpg'}}]

In [76]:
result['hits']['hits'][0]['_source']

10

In [79]:
list(result['hits']['hits'][0]['_source'].items())[1][1]

'patterned_blouse_106.jpg'